In [1]:
import faiss
import math
!pip install termcolor > /dev/null
!pip install langchain
!pip install openai
!pip install langchain_experimental
!pip install tiktoken
!pip install faiss-cpu

In [21]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory
)
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from termcolor import colored
from langchain.vectorstores import FAISS

LLM = ChatOpenAI(max_tokens=1500)

def relevance_score_fn(score: float):
    """Returns a similarity score on a scale [0,1]"""
    return 1.0 - score / math.sqrt(2)

def create_new_memory_retriever():
    embeddings_model = OpenAIEmbeddings()
    embeddings_size = 1536
    index = faiss.IndexFlatL2(embeddings_size)
    vector_store = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn = relevance_score_fn
    )
    return TimeWeightedVectorStoreRetriever(vectorstore= vector_store, other_score_keys = ['importance'],k=15)


agent_memory = GenerativeAgentMemory(
    llm = LLM,
    memory_retriever = create_new_memory_retriever(),
    verbose =False,
    reflection_threshold = 8
)

me = GenerativeAgent(
    name = "me",
    age=38,
    traits = "curious, kind, creative",
    status = "helpful",
    memory_retriever = create_new_memory_retriever(),
    llm= LLM,
    memory = agent_memory
)




`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [22]:
me_observations = [
    "I am a curious person",
    "I am a kind person",
    "I am a creative person",
    "I am a helpful person"
]

for observation in me_observations:
    me.memory.add_memory(observation)

print(me.get_summary(force_refresh=True))

Name: me (age: 38)
Innate traits: curious, kind, creative
Creative, helpful, curious, kind.


In [23]:
USER_NAME = "Me"
def interview_agent(agent: GenerativeAgent,message: str):
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]
    

In [24]:
interview_agent(me,"What do you like to do?")

'me said "I enjoy painting, reading, and going on hikes. How about you?"'

In [26]:
me_daily_observations = [
    "I starts my day with a cup of coffee",
    "I starts my day studying in the morning",
    "During the working hours, I am focused on my work",
]

for observation in me_daily_observations:
    me.memory.add_memory(observation)
    

In [27]:
for i,observation in enumerate(me_daily_observations):
    _, reaction = me.generate_reaction(observation)
    print(colored(observation,"green"),reaction)
    if ((i+1) % len(me_daily_observations)) == 0:
        print("*" * 40)
        print(
            colored(
                f"After these observations, Me summary is: \n{me.get_summary(force_refresh=True)}","blue"
            )
        )
        print("*" * 40)

I starts my day with a cup of coffee me said "I also enjoy starting my day with a cup of coffee, it helps me feel more awake and focused."
I starts my day studying in the morning me It's great to see I starting the day off with studying, dedication to learning is admirable.
During the working hours, I am focused on my work me It's great to see I staying focused on their work during the working hours.
****************************************
After these observations, Me summary is: 
Name: me (age: 38)
Innate traits: curious, kind, creative
The core characteristics of me based on the provided statements are that I am a creative, helpful, curious, kind, and focused individual who enjoys painting, reading, going on hikes, starting the day with a cup of coffee, and studying in the morning.
****************************************
